In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gauravsharma99/fer13-cleaned-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fer13-cleaned-dataset


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jangedoo/utkface-new")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/utkface-new


In [ ]:


UTK_PATH = "/kaggle/input/utkface-new/UTKFace"
IMG_SIZE = 128


In [ ]:
pip install opencv-python

In [ ]:
import os
import cv2
import numpy as np

UTK_PATH = "/kaggle/input/utkface-new/UTKFace"
IMG_SIZE = 128

images, ages, ethnicities = [], [], []

for img_name in os.listdir(UTK_PATH):
    try:
        age, gender, ethnicity, _ = img_name.split("_")
        img_path = os.path.join(UTK_PATH, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        images.append(img)
        ages.append(int(age))
        ethnicities.append(int(ethnicity))  # 0–4 classes
    except:
        continue

X_utk = np.array(images) / 255.0
y_age = np.array(ages)
y_ethnicity = np.array(ethnicities)

print("UTKFace:", X_utk.shape, y_age.shape, y_ethnicity.shape)


UTKFace: (23705, 128, 128, 3) (23705,) (23705,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths
sorted_path = "/kaggle/working/utkface_sorted"

# Data generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    sorted_path,
    target_size=(128,128),
    batch_size=64,
    class_mode="sparse",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    sorted_path,
    target_size=(128,128),
    batch_size=64,
    class_mode="sparse",
    subset="validation"
)

# Simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation="softmax")  # 5 ethnicities
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

# Save model
model.save("/kaggle/working/utkface_ethnicity_model.h5")
print("✅ Model saved at /kaggle/working/utkface_ethnicity_model.h5")


Found 18966 images belonging to 5 classes.
Found 4739 images belonging to 5 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


297/297 ━━━━━━━━━━━━━━━━━━━━ 30s 81ms/step - accuracy: 0.4900 - loss: 1.3334 - val_accuracy: 0.6624 - val_loss: 0.9523
Epoch 2/10
297/297 ━━━━━━━━━━━━━━━━━━━━ 19s 65ms/step - accuracy: 0.6930 - loss: 0.8779 - val_accuracy: 0.6991 - val_loss: 0.8613
Epoch 3/10
297/297 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - accuracy: 0.7359 - loss: 0.7628 - val_accuracy: 0.7092 - val_loss: 0.8380
Epoch 4/10
297/297 ━━━━━━━━━━━━━━━━━━━━ 20s 67ms/step - accuracy: 0.7578 - loss: 0.6902 - val_accuracy: 0.7234 - val_loss: 0.7908
Epoch 5/10
297/297 ━━━━━━━━━━━━━━━━━━━━ 19s 64ms/step - accuracy: 0.7813 - loss: 0.6295 - val_accuracy: 0.7356 - val_loss: 0.7515
Epoch 6/10
297/297 ━━━━━━━━━━━━━━━━━━━━ 20s 66ms/step - accuracy: 0.7982 - loss: 0.5939 - val_accuracy: 0.7464 - val_loss: 0.7332
Epoch 7/10
297/297 ━━━━━━━━━━━━━━━━━━━━ 20s 67ms/step - accuracy: 0.8095 - loss: 0.5485 - val_accuracy: 0.7449 - val_loss: 0.7353
Epoch 8/10
297/297 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - accuracy: 0.8219 - loss: 0.5202 - val_accurac

✅ Model saved at /kaggle/working/utkface_ethnicity_model.h5


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_age_train, y_age_test, y_eth_train, y_eth_test = train_test_split(
    X_utk, y_age, y_ethnicity, test_size=0.2, random_state=42
)

print("Train:", X_train.shape, " Test:", X_test.shape)


In [ ]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input

input_layer = Input(shape=(128,128,3))

# Shared CNN layers
x = layers.Conv2D(32, (3,3), activation="relu")(input_layer)
x = layers.MaxPooling2D(2,2)(x)

x = layers.Conv2D(64, (3,3), activation="relu")(x)
x = layers.MaxPooling2D(2,2)(x)

x = layers.Conv2D(128, (3,3), activation="relu")(x)
x = layers.MaxPooling2D(2,2)(x)

x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# Branch 1: Age (Regression)
age_output = layers.Dense(1, activation="linear", name="age_output")(x)

# Branch 2: Ethnicity (Classification)
ethnicity_output = layers.Dense(5, activation="softmax", name="ethnicity_output")(x)

# Build model
multi_model = models.Model(inputs=input_layer, outputs=[age_output, ethnicity_output])

multi_model.compile(
    optimizer="adam",
    loss={
        "age_output": "mae",  # Regression loss
        "ethnicity_output": "sparse_categorical_crossentropy"  # Classification loss
    },
    metrics={
        "age_output": ["mae"],
        "ethnicity_output": ["accuracy"]
    }
)

multi_model.summary()


In [ ]:
history = multi_model.fit(
    X_train,
    {"age_output": y_age_train, "ethnicity_output": y_eth_train},
    validation_data=(X_test, {"age_output": y_age_test, "ethnicity_output": y_eth_test}),
    epochs=15,
    batch_size=64
)

multi_model.save("/kaggle/working/age_ethnicity_model.h5")
print("✅ Saved model at /kaggle/working/age_ethnicity_model.h5")


Epoch 1/20
297/297 ━━━━━━━━━━━━━━━━━━━━ 486s 2s/step - age_output_loss: 8.9546 - age_output_mae: 8.9546 - ethnicity_output_accuracy: 0.6180 - ethnicity_output_loss: 1.0267 - loss: 9.9813 - val_age_output_loss: 7.8169 - val_age_output_mae: 7.8229 - val_ethnicity_output_accuracy: 0.6703 - val_ethnicity_output_loss: 0.8936 - val_loss: 8.7133
Epoch 2/20
297/297 ━━━━━━━━━━━━━━━━━━━━ 483s 2s/step - age_output_loss: 8.4936 - age_output_mae: 8.4936 - ethnicity_output_accuracy: 0.6438 - ethnicity_output_loss: 0.9772 - loss: 9.4709 - val_age_output_loss: 7.6635 - val_age_output_mae: 7.6511 - val_ethnicity_output_accuracy: 0.7089 - val_ethnicity_output_loss: 0.8216 - val_loss: 8.4695
Epoch 3/20
297/297 ━━━━━━━━━━━━━━━━━━━━ 485s 2s/step - age_output_loss: 8.1288 - age_output_mae: 8.1288 - ethnicity_output_accuracy: 0.6579 - ethnicity_output_loss: 0.9484 - loss: 9.0772 - val_age_output_loss: 7.5612 - val_age_output_mae: 7.5564 - val_ethnicity_output_accuracy: 0.7258 - val_ethnicity_output_loss: 0.8

In [ ]:
import numpy as np
import cv2

def predict_age_ethnicity(model, img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128,128))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    age_pred, eth_pred = model.predict(img)
    age = int(age_pred[0][0])
    ethnicity = np.argmax(eth_pred[0])

    return age, ethnicity

# Example
age, ethnicity = predict_age_ethnicity(multi_model, os.path.join(UTK_PATH, os.listdir(UTK_PATH)[0]))
print("Predicted Age:", age, "Predicted Ethnicity:", ethnicity)


In [ ]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

FER_PATH = "/kaggle/input/fer13-cleaned-dataset"

IMG_SIZE = 48  # FER dataset images are usually 48x48

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    FER_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",   # FER is grayscale
    batch_size=64,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    FER_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=64,
    class_mode="categorical",
    subset="validation"
)

num_classes = train_gen.num_classes
print("Number of emotion classes:", num_classes)


Found 13502 images belonging to 5 classes.
Found 3374 images belonging to 5 classes.
Number of emotion classes: 5


In [ ]:
from tensorflow.keras import layers, models

emotion_model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])

emotion_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

emotion_model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 21, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 619,397 (2.36 MB)

 Trainable params: 618,949 (2.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
history_emotion = emotion_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30
)

emotion_model.save("/kaggle/working/emotion_model.h5")
print("✅ Emotion model saved at /kaggle/working/emotion_model.h5")


Epoch 1/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 82s 391ms/step - accuracy: 0.6973 - loss: 0.7783 - val_accuracy: 0.6867 - val_loss: 0.8164
Epoch 2/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 84s 396ms/step - accuracy: 0.7150 - loss: 0.7293 - val_accuracy: 0.6858 - val_loss: 0.8474
Epoch 3/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 84s 398ms/step - accuracy: 0.7203 - loss: 0.7154 - val_accuracy: 0.6929 - val_loss: 0.7975
Epoch 4/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 82s 390ms/step - accuracy: 0.7249 - loss: 0.7114 - val_accuracy: 0.6814 - val_loss: 0.8660
Epoch 5/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 81s 385ms/step - accuracy: 0.7403 - loss: 0.6762 - val_accuracy: 0.6995 - val_loss: 0.8500
Epoch 6/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 83s 393ms/step - accuracy: 0.7386 - loss: 0.6812 - val_accuracy: 0.6651 - val_loss: 0.8826
Epoch 7/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 83s 392ms/step - accuracy: 0.7519 - loss: 0.6426 - val_accuracy: 0.6657 - val_loss: 0.8682
Epoch 8/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 82s 390ms/step - accuracy: 0.7593 - loss: 0

✅ Emotion model saved at /kaggle/working/emotion_model.h5


In [ ]:
from google.colab import files
files.download("/kaggle/working/emotion_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 19.0 MB/s eta 0:00:00


In [ ]:
import cv2

def predict_emotion(model, img_path, class_indices):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=-1)  # add channel
    img = np.expand_dims(img, axis=0)   # add batch

    pred = model.predict(img)
    emotion_idx = np.argmax(pred[0])

    labels = {v: k for k, v in class_indices.items()}
    return labels[emotion_idx]

# Example usage
emotion = predict_emotion(emotion_model, os.path.join(FER_PATH,"Angry/Training_10017485.jpg"), train_gen.class_indices)
print("Predicted Emotion:", emotion)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted Emotion: Angry
